<img src='http://www.u.arizona.edu/~hammond/hltlogo1.png' style="float:right">

Linguistics 578<br>
Fall 2022<br>
Hammond

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted.
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Please do not add, remove, or copy autograded cells.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

***

***my name***: Rachel Hansen

***people I talked to about the assignment***: [put your answer here]

***

## Homework #7

The focus of this assignment is *state tying*. This is actually fairly simple with `pomegranate`. When you create a state, you specify some sort of distribution. If you use the same distribution for multiple states, as opposed to *copying* the distribution or making a new one, then the emissions from those states are tied.


Here are the imports. Please do not import anything else.

In [1]:
import pomegranate as p
import numpy as np

1. Let's first create a discrete distribution over the symbols `a` and `b`. Specify their probabilities as $.3$ and $.7$ respectively.

In [2]:
# d = ?
d = p.DiscreteDistribution({'a':.3,'b':.7})
#raise NotImplementedError()

In [3]:
assert d.parameters[0] == {'b':.7,'a':.3}

2. Now create an HMM with two states `s1` and `s2`. The only start state is `s1` and the only final state is `s2`. You also need arcs from `s1` to `s2` ($1.0$) and another from `s2` to `s2` ($0.5$). Use `d` above as the emissions distribution for `s1` and use a *copy* of `d` for `s2`.

In [4]:
# hmm1 = ?
model = p.HiddenMarkovModel()
s1 = p.State(d)
s2 = p.State(d.copy())
model.add_states(s1, s2)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s2, 1.0)
model.add_transition(s2, s2, 0.5)
model.add_transition(s2, model.end, 0.5)
model.bake()
hmm1 = model
#raise NotImplementedError()

In [5]:
assert type(hmm1) == p.hmm.HiddenMarkovModel

In [6]:
assert (hmm1.dense_transition_matrix() == np.array([
    [0. , 1. , 0. , 0. ],
    [0. , 0.5, 0. , 0.5],
    [1. , 0. , 0. , 0. ],
    [0. , 0. , 0. , 0. ]
])).all()

In [7]:
ab = hmm1.probability('ab')
assert np.isclose(ab,0.105,atol=.005)

In [8]:
assert hmm1.states[0].distribution.parameters[0] == {'a': 0.3, 'b': 0.7} \
and hmm1.states[1].distribution.parameters[0] == {'a': 0.3, 'b': 0.7}

In [9]:
train = [np.array(list(word)) for word in ['ab','aaa','bba']]
hmm1.fit(train)
assert hmm1.states[0].distribution.parameters[0] != \
hmm1.states[1].distribution.parameters[0]

In [10]:
assert hmm1.probability('ab') > ab

3. Now make a new HMM just like the one above *except* that the two states share a single new distribution specified so that `a` is `.4` and `b` is `.6`. (Make sure to name this distribution differently from the distribution names you used above.)

In [11]:
# hmm2 = ?
d2 = p.DiscreteDistribution({'a':.4,'b':.6})
model = p.HiddenMarkovModel()
s1 = p.State( d2, name="Tied1" )
s2 = p.State( d2, name="Tied2" )
model.add_states(s1, s2)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s2, 1.0)
model.add_transition(s2, s2, 0.5)
model.add_transition(s2, model.end, 0.5)
model.bake()
hmm2 = model
#raise NotImplementedError()

In [12]:
assert hmm2.states[0].distribution.parameters[0] == \
hmm2.states[1].distribution.parameters[0] == \
{'a': 0.4, 'b': 0.6}

In [13]:
ba = hmm2.probability('ba')
assert np.isclose(ba,.12,atol=.001)

In [14]:
train2 = [np.array(list(word)) for word in ['ab','aa','bba']]
hmm2.fit(train2)
assert hmm2.states[0].distribution.parameters[0] == \
hmm2.states[1].distribution.parameters[0]

In [15]:
assert hmm2.probability('ba') > ba

4. We'll now create two HMMs: `hmm3` and `hmm4`. Both will have two nodes `s1` and `s2`, and both will have the same linear structure. The only start nodes are `s1` and the only final nodes are `s2`. There are self-loops on both states and a single transition from `s1` to `s2`. The initial emissions probabilities for all states are $.5$ for `a` and `b`. Initial arc probabilities are all $0.5$. Emission probabilities are tied for `s1` in `hmm3` and `s2` in `hmm4`.

In [16]:
# hmm3 = ?
# hmm4 = ?
d3 = p.DiscreteDistribution({'a':.5,'b':.5})
model = p.HiddenMarkovModel()
s1 = p.State( d3, name="Tied1" )
s2 = p.State( d3.copy() )
model.add_states(s1, s2)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s2, 0.5)
model.add_transition(s1, s1, 0.5)
model.add_transition(s2, s2, 0.5)
model.add_transition(s2, model.end, 0.5)
model.bake()
hmm3 = model

model = p.HiddenMarkovModel()
s1 = p.State( d3.copy())
s2 = p.State( d3, name="Tied2" )
model.add_states(s1, s2)
model.add_transition(model.start, s1, 1.0)
model.add_transition(s1, s2, 0.5)
model.add_transition(s1, s1, 0.5)
model.add_transition(s2, s2, 0.5)
model.add_transition(s2, model.end, 0.5)
model.bake()
hmm4 = model

#raise NotImplementedError()

In [17]:
assert hmm3.probability('abb') == hmm4.probability('abb')

In [18]:
assert hmm3.states[0].distribution.parameters[0] == \
hmm3.states[1].distribution.parameters[0] == \
hmm4.states[0].distribution.parameters[0] == \
hmm4.states[1].distribution.parameters[0]

In [19]:
train3 = [np.array(list(word)) for word in ['abb','aa','bba']]
hmm3.fit(train3)
assert hmm3.states[0].distribution.parameters[0] != \
hmm3.states[1].distribution.parameters[0]

In [20]:
assert hmm3.states[0].distribution.parameters[0] == \
hmm4.states[1].distribution.parameters[0]